In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)

### Import Datasets

In [2]:
mortality_2003_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2003.txt', sep='\t')
mortality_2004_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2004.txt', sep='\t')
mortality_2005_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2005.txt', sep='\t')
mortality_2006_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2006.txt', sep='\t')
mortality_2007_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2007.txt', sep='\t')
mortality_2008_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2008.txt', sep='\t')
mortality_2009_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2009.txt', sep='\t')
mortality_2010_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2010.txt', sep='\t')
mortality_2011_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2011.txt', sep='\t')
mortality_2012_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2012.txt', sep='\t')
mortality_2013_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2013.txt', sep='\t')
mortality_2014_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2014.txt', sep='\t')
mortality_2015_Df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/US_VitalStatistics/Underlying Cause of Death, 2015.txt', sep='\t')

mortality_list = [mortality_2003_Df,
mortality_2004_Df,
mortality_2005_Df,
mortality_2006_Df,
mortality_2007_Df,
mortality_2008_Df,
mortality_2009_Df,
mortality_2010_Df,
mortality_2011_Df,
mortality_2012_Df,
mortality_2013_Df,
mortality_2014_Df,
mortality_2015_Df]

mortality = pd.DataFrame()
for i in mortality_list:
    mortality = mortality.append(i)

mortality = mortality.reset_index()

opioid_df = pd.read_csv('/Users/ashish/Desktop/Duke/Fall 2020/IDS 720/estimating-impact-of-opioids-2020-vxf/00_source/arcos_all_washpost.tsv'
                        , sep='\t'
                        , nrows = 5000)



### Drop counties that contain 'missing' keyword in Deaths column

In [3]:
mortality[mortality['Deaths'] == 'Missing']['County'].value_counts()

Prince of Wales-Outer Ketchikan Census Area, AK    8
Bedford city, VA                                   8
Clifton Forge city, VA                             8
Wrangell-Petersburg Census Area, AK                8
Skagway-Hoonah-Angoon Census Area, AK              8
Name: County, dtype: int64

In [4]:
mortality = mortality[mortality['Deaths'] != 'Missing']

In [5]:
mortality[mortality['Deaths'] == 'Missing']['County'].value_counts()

Series([], Name: County, dtype: int64)

In [6]:
mortality['Deaths'] = pd.to_numeric(mortality['Deaths'])

In [7]:
mortality.head()

,index,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,0,NaN,"Autauga County, AL",1001.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0
1,1,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0
2,2,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,All other alcohol-induced causes,A9,14.0
3,3,NaN,"Baldwin County, AL",1003.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0
4,4,NaN,"Barbour County, AL",1005.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0


### Identify Drug Related Types
1. Drug poisonings (overdose) Unintentional (X40-X44) <br>
2. Drug poisonings (overdose) Suicide (X60-X64) <br>
3. Drug poisonings (overdose) Undetermined (Y10-Y14) <br>
4. All other drug-induced causes <br>
5. Drug poisonings (overdose) Homicide (X85)

In [8]:
mortality['Drug/Alcohol Induced Cause'].value_counts()

All other non-drug and non-alcohol causes             40332
Drug poisonings (overdose) Unintentional (X40-X44)     7568
All other alcohol-induced causes                       6118
Drug poisonings (overdose) Suicide (X60-X64)           1460
Drug poisonings (overdose) Undetermined (Y10-Y14)       756
All other drug-induced causes                           623
Alcohol poisonings (overdose) (X45, X65, Y15)           344
Name: Drug/Alcohol Induced Cause, dtype: int64

In [9]:
subset_list = ['Drug poisonings (overdose) Unintentional (X40-X44)',
              'Drug poisonings (overdose) Suicide (X60-X64)',
              'Drug poisonings (overdose) Undetermined (Y10-Y14)',
              'All other drug-induced causes',
              'Drug poisonings (overdose) Homicide (X85)']

In [10]:
mortality_sub = mortality[mortality['Drug/Alcohol Induced Cause'].isin(subset_list)]

In [11]:
mortality_sub = mortality_sub[['County', 'Year', 'Deaths']]
mortality_sub.head()

,County,Year,Deaths
1,"Baldwin County, AL",2003.0,10.0
38,"Jefferson County, AL",2003.0,37.0
39,"Jefferson County, AL",2003.0,32.0
54,"Mobile County, AL",2003.0,26.0
78,"Anchorage Borough, AK",2003.0,31.0


In [12]:
mortality_sub = mortality_sub.groupby(['County', 'Year'], as_index=False).sum()

In [13]:
mortality_sub.head()

,County,Year,Deaths
0,"Acadia Parish, LA",2003.0,11.0
1,"Acadia Parish, LA",2005.0,23.0
2,"Acadia Parish, LA",2006.0,19.0
3,"Acadia Parish, LA",2007.0,19.0
4,"Acadia Parish, LA",2009.0,11.0


### Parse out State and County from County column

In [14]:
state = [x[-2:] for x in mortality_sub['County'].tolist()]
county = [x[:-4] for x in mortality_sub['County'].tolist()]
d = pd.DataFrame({'State':state, 'County':county})
d.head()

,State,County
0,LA,Acadia Parish
1,LA,Acadia Parish
2,LA,Acadia Parish
3,LA,Acadia Parish
4,LA,Acadia Parish


In [15]:
mortality_sub = mortality_sub.merge(d, how='inner', left_index=True, right_index=True)
mortality_sub = mortality_sub.drop('County_x', axis=1)
mortality_sub.head()

,Year,Deaths,State,County_y
0,2003.0,11.0,LA,Acadia Parish
1,2005.0,23.0,LA,Acadia Parish
2,2006.0,19.0,LA,Acadia Parish
3,2007.0,19.0,LA,Acadia Parish
4,2009.0,11.0,LA,Acadia Parish


In [16]:
mortality_sub[mortality_sub['Year']==2015.0]

,Year,Deaths,State,County_y
19,2015.0,44.0,ID,Ada County
32,2015.0,84.0,CO,Adams County
33,2015.0,14.0,IL,Adams County
36,2015.0,13.0,OH,Adams County
51,2015.0,30.0,SC,Aiken County
...,...,...,...,...
7870,2015.0,26.0,CA,Yolo County
7883,2015.0,39.0,ME,York County
7896,2015.0,84.0,PA,York County
7909,2015.0,38.0,SC,York County


In [17]:
mortality_sub = mortality_sub[['Year', 'State', 'County_y', 'Deaths']]
mortality_sub.columns = ['Year', 'State', 'County', 'Deaths']
mortality_sub.head()

,Year,State,County,Deaths
0,2003.0,LA,Acadia Parish,11.0
1,2005.0,LA,Acadia Parish,23.0
2,2006.0,LA,Acadia Parish,19.0
3,2007.0,LA,Acadia Parish,19.0
4,2009.0,LA,Acadia Parish,11.0


In [18]:
mortality_sub['Year'].dtype

dtype('float64')

### Aggregate to State Level

In [24]:
mortality_sub_group = mortality_sub[['Year','State', 'Deaths']]
mortality_state = mortality_sub_group.groupby(['Year', 'State'], as_index=False).sum()
mortality_state.head()

,Year,State,Deaths
0,2003.0,AK,42.0
1,2003.0,AL,105.0
2,2003.0,AR,18.0
3,2003.0,AZ,640.0
4,2003.0,CA,3241.0


### Drop Alaska as per requirements

In [33]:
mortality_state = mortality_state[mortality_state['State'] != 'AL']
mortality_state.head()

,Year,State,Deaths
0,2003.0,AK,42.0
2,2003.0,AR,18.0
3,2003.0,AZ,640.0
4,2003.0,CA,3241.0
5,2003.0,CO,361.0


### Output data to csv

In [34]:
mortality_state.to_csv('../20_intermediate_files/mortality.csv')

### Plot Data to find similar trends